In [1]:
from pyexcel_xlsx import get_data

In [2]:
print("reading data")
data = get_data("spanglish_pos.xlsx", encoding='utf-8')["spanglish"]
texts = []
labels = []
pos_tags = []


reading data


In [2]:
import io
with io.open("spanglish_lt4.txt", "r", encoding="utf-8") as f: 
    spline = [line.strip('\n').split(";") for line in f]


In [3]:
import io
texts = []
labels = []
for i in range(len(spline)): 
    texts.append(spline[i][0])
    labels.append(spline[i][1])


In [4]:
print(len(texts))
print(len(labels))
set(labels)

8011
8011


{'eng', 'span'}

In [5]:
punct = {"!", "?", ".", "-","UNK", ","}
i = 0
#print(texts[i] in punct)
while(i in range(len(texts))): 
    if(texts[i] == "<unk>"): 
        labels[i] = "<unk>"
    if(texts[i]) in punct:
        labels[i] = "punct"
        #print("TRUE") 
    #else: 
        #print("FALSE")
        #print(labels[i])
    i += 1
        

       


In [6]:
set(labels)

{'<unk>', 'eng', 'punct', 'span'}

In [1]:

i = 0
with io.open("Span-Eng.txt", "w", encoding="utf-8") as out: 
    i = 0
    for i in range(len(texts)): 
        print(texts[i] + "|" + labels[i], file=out)
        i += 1

NameError: name 'io' is not defined

In [12]:
with io.open("Trigrams2.txt", "r", encoding="utf-8") as f: 
    trigrams = [line.strip("\n") for line in f]

In [19]:
trigrams[:10]
#print(len(trigrams))

['$ o h #',
 '$ , #',
 '$ n o #',
 '$ . #',
 '$ i #',
 '$ a m #',
 '$ n o t #',
 'gon onn nna',
 'tak ake',
 '$ h e r #']

In [20]:
i = 0
for i in range(len(texts)): 
    texts[i] = trigrams[i]

In [14]:
texts[:10]

['oh', ',', 'no', '.', 'i', 'am', 'not', 'gonna', 'take', 'her']

In [7]:
# Split data into train and test
train_size = int(len(texts) * .7)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(texts) - train_size))

Train size: 5607
Test size: 2404


In [48]:
print(set(ytrain))
set(ytest)

{'<unk>', 'span', 'punct', 'eng'}


{'<unk>', 'eng', 'punct', 'span'}

In [10]:
tokenizer = Tokenizer(num_words=5)
toytexts = ["Is is a common word", "So is the", "the is common", "discombobulation is not common"]
tokenizer.fit_on_texts(toytexts)
sequences = tokenizer.texts_to_sequences(toytexts)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [23]:
xtrain = tfidf_matrix[:train_size]
xtest = tfidf_matrix[train_size:]
ytrain = labels[:train_size]
ytest = labels[train_size:]

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [50]:
tokenize.fit_on_texts(xtrain) # only fit on train
x_train = tokenize.texts_to_matrix(xtrain, mode='tfidf')
x_test = tokenize.texts_to_matrix(xtest, mode='tfidf')

In [24]:
# Use sklearn utility to convert label strings to numbered index
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
print('Encoding labels...')
encoder = LabelEncoder()
encoder.fit(labels)
ytrain = encoder.transform(ytrain)
ytest = encoder.transform(ytest)


Encoding labels...


In [26]:
xtrain.shape

(5607, 1277)

In [28]:
# Converts the labels to a one-hot representation
from keras import utils
import numpy as np
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [29]:
y_train[1]

array([ 0.,  0.,  1.,  0.])

In [33]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', xtrain.shape)
print('x_test shape:', xtest.shape)
print('y_train shape:', ytrain.shape)
print('y_test shape:', ytest.shape)

x_train shape: (5607, 1277)
x_test shape: (2404, 1277)
y_train shape: (5607,)
y_test shape: (2404,)


In [34]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 10

In [46]:
# Build the model
from keras.layers.normalization import BatchNormalization
from keras.layers.core import *
from keras.regularizers import l2
from keras import optimizers

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
model = Sequential()
model.add(Dense(input_shape=tfidf_matrix.shape[1]+2, init='normal', activation='relu'))
#model.add(Dense(512, input_shape=(max_words,)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Dropout(0.4))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Dropout(0.4))
model.add(Activation('sigmoid'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_initializer="normal", input_shape=1279)`
  if __name__ == '__main__':


TypeError: __init__() missing 1 required positional argument: 'units'

In [40]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(xtrain, ytrain,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)

ValueError: Error when checking input: expected dense_3_input to have shape (None, 5607) but got array with shape (5607, 1277)

In [72]:
score = model.evaluate(x_test, y_test, verbose=1, batch_size=batch_size)
                       #batch_size=batch_size, verbose=0)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])
print(score)

2080/2404 [========================>.....] - ETA: 0s[1.4540465751295009, 0.78577371048252909]


In [22]:
import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

6408


In [49]:
print(len(xtest))

1603


In [53]:
print('xtrain shape:', xtrain.shape)
print('xtest shape:', xtest.shape)


xtrain shape: (6408, 1288)
xtest shape: (1603, 1288)


In [55]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
print('Encoding labels...')
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(ytrain)
integer_encoded2 = label_encoder.fit_transform(ytest)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
integer_encoded2 = integer_encoded.reshape(len(integer_encoded2), 1)
ytrain = integer_encoded
ytest = integer_encoded2
from keras.utils import np_utils
ytrain = keras.utils.to_categorical(ytrain)
ytest = keras.utils.to_categorical(ytest)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
Encoding labels...


In [67]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(xtrain.shape[0],)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Building model...


In [68]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [65]:
batch_size = 32
epochs = 5

history = model.fit(xtrain, ytrain,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

ValueError: Error when checking input: expected dense_1_input to have shape (None, 1000) but got array with shape (6408, 1288)